In [758]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [761]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [760]:
# skip if result.txt exists
y16 = glob.glob('data/2016'+'/*.txt')
y15 = glob.glob('data/2015'+'/*.txt')
y14 = glob.glob('data/2014'+'/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==9:
                    outfile.write(line+'\n')

In [762]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author']
df.shape

(9043461, 9)

In [626]:
df=df[df['author']==1]
df.shape

(8441035, 9)

In [410]:
df['year']=df['year'].astype(int)

In [411]:
df=df[df['year']<=2017][df['year']>=1950]
df.shape

(8441035, 9)

In [675]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [676]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(8436442, 12)

In [677]:
df=df[['count','cited','citation','issnp','issne','journal']].fillna('')

In [732]:
issnps_all=df.issnp.values
df.loc[(df['issne'].isin(issnps_all)) & (df['issnp']==''),'issnp']=df['issne']

In [733]:
issnps=df[(df.issne!='') & (df.issnp!='')].issnp.values
issnes=df[(df.issne!='') & (df.issnp!='')].issne.values

In [734]:
p={}
i=0
for issnp in issnps:
    p[issnp]=issnes[i]
    i=i+1
e={}
i=0
for issne in issnes:
    e[issne]=issnps[i]
    i=i+1

In [735]:
df['issnp2']=df.apply(lambda x: e[x['issne']] if x['issne'] in e else x['issnp'],axis=1)
df['issne2']=df.apply(lambda x: p[x['issnp']] if x['issnp'] in p else x['issne'],axis=1)

In [736]:
dg=df.groupby(['issnp2','issne2']).agg({'count':sum,'cited':sum,'citation':sum,'journal':max}).reset_index()
dg.shape

(39597, 6)

In [737]:
dg2=df[['issnp2','issne2','citation']].groupby(['issnp2','issne2']).agg(lambda x:tuple(x)).reset_index()
dg['h']=dg2.citation.apply(lambda x: h_(x))

In [738]:
dg.sort_values('citation',ascending=False).head()

,issnp2,issne2,count,cited,citation,journal,h
2489,,2045-2322,36016,19899,140079,Scientific Reports,65
9588,0028-4793,1533-4406,2599,2170,123217,New England Journal of Medicine,178
9566,0028-0836,1476-4687,9399,3027,122663,Nature,161
1878,,1932-6203,84715,13301,111701,PLoS ONE,60
10056,0036-8075,1095-9203,13214,2552,109775,Science,156


In [739]:
dg['impact']=dg['citation']/dg['count']
dg.sort_values('impact',ascending=False).head()

,issnp2,issne2,count,cited,citation,journal,h,impact
8277,0007-9235,,116,44,18297,CA: A Cancer Journal for Clinicians,18,157.732759
37835,2367-3613,1433-8351,13,10,939,Living Reviews in Relativity,7,72.230769
8354,0009-2665,1520-6890,910,494,58230,Chemical Reviews,144,63.989011
13544,0306-0012,1460-4744,1064,487,53390,Chemical Society Reviews,134,50.178571
18410,1057-5987,1545-8601,8,7,390,MMWR. Recommendations and Reports,6,48.750000


In [740]:
dg['@id']=dg.apply(lambda x:x['issne2'] if x['issnp2']=='' else x['issnp2'],1)

In [741]:
df[df['issnp']=='2229-0443']
dg[dg['@id']=='2229-0443']

,issnp2,issne2,count,cited,citation,journal,h,impact,@id
34934,2229-0443,2229-0443,30,2,5,Language Testing in Asia,1,0.166667,2229-0443


In [742]:
dic=[]
for index, i in dg.iterrows():
    journal={}
    journal['@id']=i['@id']
    issn_list=[i['@id']]
    if (i['issne2']!='') and (i['issne2']!=i['@id']):
        issn_list.append(i['issne2'])
    journal['e:issn']=issn_list
    journal['e:count']=i['count']
    journal['e:cited']=i['cited']
    journal['e:citation']=i['citation']
    journal['e:h']=i['h']
    journal['e:impact']=round(i['impact'],2)
    if journal not in dic:
        dic.append(journal)

In [743]:
with open('journals_stats.jsonld', 'w') as outfile:
    json.dump(dic, outfile)

In [757]:
df[df['issnp']=='1047-7047'].sort_values('citation',ascending=False)

,doi,year,ref,citation,issnp,issne,journal,publisher,author
3296101,10.1287/isre.2014.0536,2014,72,55,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296111,10.1287/isre.2014.0546,2014,13,54,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296068,10.1287/isre.2013.0501,2014,114,33,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296073,10.1287/isre.2013.0508,2014,70,19,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296100,10.1287/isre.2014.0535,2014,140,16,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296102,10.1287/isre.2014.0537,2014,109,16,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296064,10.1287/isre.2013.0493,2014,46,12,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296154,10.1287/isre.2015.0588,2015,169,11,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296108,10.1287/isre.2014.0543,2014,18,10,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1
3296066,10.1287/isre.2013.0496,2014,114,9,1047-7047,1526-5536,Information Systems Research,Institute for Operations Research and the Mana...,1


In [754]:
df.head()

,doi,year,ref,citation,issnp,issne,journal,publisher,author
0,10.1001/2013.jamasurg.124,2014,0,0,2168-6254,NaN,JAMA Surgery,American Medical Association (AMA),1
1,10.1001/2013.jamasurg.273,2014,0,0,2168-6254,NaN,JAMA Surgery,American Medical Association (AMA),1
2,10.1001/2013.jamasurg.351,2014,0,0,2168-6254,NaN,JAMA Surgery,American Medical Association (AMA),1
3,10.1001/2013.jamasurg.356,2014,0,0,2168-6254,NaN,JAMA Surgery,American Medical Association (AMA),1
4,10.1001/jama.2013.133752,2014,0,0,0098-7484,NaN,JAMA,American Medical Association (AMA),1
